In [1]:
from run_train import run_train
from run_test import run_test, show_attn_align, get_voice, compare_wav_plot
from network import Tacotron
from utils import TacotronDataset, collate_fn
import hyperparameter as hp

import torch
from torch.utils.data import DataLoader
from transformers import AdamW
import numpy as np
import random
from matplotlib import pyplot as plt

C:\Users\leejo\Desktop\파이썬\venv-bert_QA_task\venv\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [ ]:
text_files = 'C:\\Users\\leejo\\Desktop\\파이썬\\tacotron\\archive\\transcript_ko.txt'
with open(text_files, "rt", encoding='UTF8') as f:
      text = f.readlines()

lin_targets = 'C:\\Users\\leejo\\Desktop\\파이썬\\tacotron\\archive\\lin_target.npy'
mel_targets = 'C:\\Users\\leejo\\Desktop\\파이썬\\tacotron\\archive\\mel_target.npy'

#check number of files(text, voice)
print("num_text: %d", len(text))
print("num_lin: %d", len(lin_targets))
print("num_mel: %d", len(mel_targets))

In [ ]:
#devide data into train, dev, test (8:1:1)
train_index = int(len(text) * 0.8)
dev_index = int(len(text) * 0.9)

text_train = text[:train_index]
text_dev = text[train_index:dev_index]
text_test = text[dev_index:]

lin_train = lin_targets[:train_index]
lin_dev = lin_targets[train_index:dev_index]
lin_test = lin_targets[dev_index:]

mel_train = mel_targets[:train_index]
mel_dev = mel_targets[train_index:dev_index]
mel_test = mel_targets[dev_index:]

In [ ]:
#load dataset
train_dataset = TacotronDataset(text_train, lin_train, mel_train)
dev_dataset = TacotronDataset(text_dev, lin_dev, mel_dev)
test_dataset = TacotronDataset(text_test, lin_test, mel_test)

train_dataloader = DataLoader(train_dataset,
                              batch_size=hp.batch_size,
                              shuffle=True,
                              collate_fn= collate_fn)
dev_dataloader = DataLoader(dev_dataset,
                              batch_size=hp.batch_size,
                              collate_fn= collate_fn)
test_dataloader = DataLoader(test_dataset,
                              batch_size=hp.batch_size,
                              collate_fn= collate_fn)

In [2]:
#load model
model = Tacotron()
model.to(hp.device)

#set optimizer
optimizer = AdamW(
    model.parameters(), 
    lr = hp.learning_rate, 
    correct_bias = False
    )
    
#random seed
seed_val = 100
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#gradient init
model.zero_grad()

In [ ]:
#test_code

#get test voice

#get test attention alignment


In [ ]:
#run_train
run_train(hp.n_epoch,
          model,
          optimizer,
          hp.device,
          train_dataloader,
          dev_dataloader)

In [3]:
#save_model
torch.save(model.state_dict(), 'C:\\Users\\leejo\\Desktop\\파이썬\\tacotron\\model\\model.pt')

SyntaxError: EOL while scanning string literal (Temp/ipykernel_14604/630936895.py, line 2)

In [ ]:
#load model
model.load_state_dict(torch.load('C:\\Users\\leejo\\Desktop\\파이썬\\tacotron\\model\\model.pt'))
model.eval()

In [ ]:
#run_test
run_test(model,
         hp.device,
         test_dataloader)

In [ ]:
#Choose random text, mel data
rand_int = random.randint(0, len(text_test)-1)
text_random = text_test[rand_int]
mel_random = mel_test[rand_int]

#get voice
voice = get_voice(model, text_random, mel_random)

#show attention alignment
show_attn_align(model, text_random, mel_random)

#compare wav plot
#대응되는 original 어떻게 가져올것인가
import librosa
files = librosa.util.find_files('archive\kss')
audio_random = files[dev_index + rand_int]
original, _ = librosa.load(audio_random, sr = hp.sample_rate)
compare_wav_plot(model, text_random, mel_random, original, hp.sample_rate)